# Part 2: Building Blocks - 构建模块 🏗️

## 🎯 学习目标

通过本教程，您将掌握：
- 📊 **状态管理** - PHMState和DAGState的设计模式
- 🔄 **LangGraph工作流** - 复杂图结构的构建和执行
- 🔀 **Router模式** - 智能路由和负载均衡
- 🏭 **Provider工厂** - 高级LLM配置和管理

## 📋 预计时长：2-3小时

---

## 🛠️ 环境准备

In [ ]:
# 导入必要的库
import sys
import os
from pathlib import Path
from typing import Dict, Any, List, TypedDict, Annotated
import operator

# 设置项目路径
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / "src"))

# 导入我们的模块
try:
    from src.states.phm_states import PHMState, DAGState, InputData
    from src.model import get_llm
    PROJECT_AVAILABLE = True
    print("✅ 使用真实项目组件")
except ImportError as e:
    print(f"⚠️ 项目模块导入失败: {e}")
    PROJECT_AVAILABLE = False
    print("🔧 将使用模拟组件")

from langgraph.graph import StateGraph, END
from langchain_core.language_models import BaseChatModel

print("✅ 环境准备完成")

---

## 1. 状态管理 - State Management 📊

### 1.1 TypedDict状态基础

在Graph Agent中，状态是整个系统的核心。让我们理解状态管理的基本概念：

In [ ]:
# 基础状态定义
class BasicState(TypedDict):
    """基础状态示例"""
    current_step: str
    messages: Annotated[List[str], operator.add]  # 自动累加列表
    sensor_data: Dict[str, float]
    analysis_results: Dict[str, Any]
    confidence: float

# 演示状态更新
def demo_state_management():
    print("🔄 状态管理演示")
    
    # 初始状态
    initial_state = {
        "current_step": "initialization",
        "messages": ["系统启动"],
        "sensor_data": {"temperature": 65.2, "vibration": 3.1},
        "analysis_results": {},
        "confidence": 0.0
    }
    
    print(f"初始状态: {initial_state}")
    
    # 状态更新1
    update1 = {
        "current_step": "analysis",
        "messages": ["开始数据分析"],  # 会自动累加到现有messages
        "analysis_results": {"frequency_domain": "正常"}
    }
    
    # 模拟LangGraph的状态合并
    merged_state = initial_state.copy()
    for key, value in update1.items():
        if key == "messages":
            merged_state[key].extend(value)  # 累加列表
        elif key == "analysis_results":
            merged_state[key].update(value)  # 更新字典
        else:
            merged_state[key] = value  # 直接替换
    
    print(f"\n更新后状态:")
    for key, value in merged_state.items():
        print(f"  {key}: {value}")

demo_state_management()

### 1.2 PHM专用状态结构

让我们学习PHM系统中使用的专门状态结构：

In [ ]:
if PROJECT_AVAILABLE:
    # 使用真实的PHMState
    print("🏭 使用真实PHMState结构")
    
    # 创建示例数据
    import numpy as np
    
    sample_signal = np.random.randn(1000).reshape(1, -1, 1)
    
    # 创建InputData节点
    input_node = InputData(
        node_id="sensor_data",
        parents=[],
        shape=sample_signal.shape,
        results={"signal": sample_signal},
        meta={"sensor_type": "accelerometer", "fs": 1000}
    )
    
    # 创建DAGState
    dag_state = DAGState(
        user_instruction="分析轴承振动信号",
        channels=["vibration_x"],
        nodes={"input": input_node},
        leaves=["input"]
    )
    
    print(f"DAG节点: {list(dag_state['nodes'].keys())}")
    print(f"用户指令: {dag_state['user_instruction']}")
    print(f"信号形状: {input_node['shape']}")
    
else:
    # 使用模拟的PHM状态
    print("🔧 使用模拟PHM状态结构")
    
    class MockPHMState(TypedDict):
        """模拟的PHM状态结构"""
        case_name: str
        user_instruction: str
        sensor_data: Dict[str, Any]
        processing_steps: Annotated[List[str], operator.add]
        analysis_results: Dict[str, Any]
        confidence_scores: Dict[str, float]
    
    # 创建示例状态
    phm_state = {
        "case_name": "bearing_diagnosis",
        "user_instruction": "分析轴承振动信号，检测潜在故障",
        "sensor_data": {
            "accelerometer": {"signal": [1, 2, 3], "fs": 1000}
        },
        "processing_steps": ["数据预处理"],
        "analysis_results": {"fft_analysis": "完成"},
        "confidence_scores": {"overall": 0.85}
    }
    
    print("模拟PHM状态创建完成:")
    for key, value in phm_state.items():
        if isinstance(value, dict) and len(str(value)) > 50:
            print(f"  {key}: {type(value).__name__} (复杂对象)")
        else:
            print(f"  {key}: {value}")

---

## 2. LangGraph工作流 - Workflow Management 🔄

### 2.1 创建多节点工作流

LangGraph允许我们创建复杂的多节点工作流：

In [ ]:
# 定义工作流状态
class WorkflowState(TypedDict):
    input_data: Dict[str, Any]
    processing_history: Annotated[List[str], operator.add]
    current_node: str
    results: Dict[str, Any]
    should_continue: bool

# 创建PHM分析工作流
class PHMAnalysisWorkflow:
    """PHM分析工作流"""
    
    def __init__(self):
        self.workflow = self._build_workflow()
    
    def _build_workflow(self) -> StateGraph:
        """构建工作流图"""
        
        def preprocessing_node(state: WorkflowState) -> Dict[str, Any]:
            """数据预处理节点"""
            print("🔄 执行数据预处理...")
            
            # 模拟预处理逻辑
            raw_data = state["input_data"].get("sensor_data", {})
            processed_data = {k: v * 1.1 for k, v in raw_data.items() if isinstance(v, (int, float))}
            
            return {
                "processing_history": ["数据预处理完成"],
                "current_node": "preprocessing",
                "results": {"processed_data": processed_data}
            }
        
        def feature_extraction_node(state: WorkflowState) -> Dict[str, Any]:
            """特征提取节点"""
            print("🔍 执行特征提取...")
            
            # 模拟特征提取
            processed_data = state["results"].get("processed_data", {})
            features = {
                "mean_value": sum(processed_data.values()) / len(processed_data) if processed_data else 0,
                "max_value": max(processed_data.values()) if processed_data else 0,
                "feature_count": len(processed_data)
            }
            
            return {
                "processing_history": ["特征提取完成"],
                "current_node": "feature_extraction",
                "results": {**state["results"], "features": features}
            }
        
        def classification_node(state: WorkflowState) -> Dict[str, Any]:
            """分类诊断节点"""
            print("🎯 执行故障诊断...")
            
            features = state["results"].get("features", {})
            mean_val = features.get("mean_value", 0)
            
            # 简单的分类逻辑
            if mean_val > 80:
                diagnosis = "严重故障"
                confidence = 0.9
            elif mean_val > 60:
                diagnosis = "轻微异常"
                confidence = 0.7
            else:
                diagnosis = "正常状态"
                confidence = 0.95
            
            return {
                "processing_history": [f"诊断完成: {diagnosis} (置信度: {confidence:.2f})"],
                "current_node": "classification",
                "results": {**state["results"], "diagnosis": diagnosis, "confidence": confidence},
                "should_continue": False
            }
        
        # 构建图
        workflow = StateGraph(WorkflowState)
        
        # 添加节点
        workflow.add_node("preprocess", preprocessing_node)
        workflow.add_node("extract_features", feature_extraction_node)
        workflow.add_node("classify", classification_node)
        
        # 定义执行顺序
        workflow.set_entry_point("preprocess")
        workflow.add_edge("preprocess", "extract_features")
        workflow.add_edge("extract_features", "classify")
        workflow.add_edge("classify", END)
        
        return workflow.compile()
    
    def analyze(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """执行分析"""
        initial_state = {
            "input_data": input_data,
            "processing_history": ["工作流启动"],
            "current_node": "start",
            "results": {},
            "should_continue": True
        }
        
        result = self.workflow.invoke(initial_state)
        return result

# 测试工作流
print("🏭 创建PHM分析工作流")
phm_workflow = PHMAnalysisWorkflow()

# 测试数据
test_data = {
    "sensor_data": {
        "temperature": 75.5,
        "vibration": 4.2,
        "pressure": 2.3
    },
    "timestamp": "2024-01-15 10:30:00"
}

print(f"\n📊 输入数据: {test_data}")
result = phm_workflow.analyze(test_data)

print(f"\n✅ 工作流执行完成")
print(f"🔄 执行历史: {result['processing_history']}")
print(f"🎯 最终诊断: {result['results'].get('diagnosis', 'N/A')}")
print(f"📈 置信度: {result['results'].get('confidence', 0):.2f}")

### 2.2 条件分支和并行处理

复杂的工作流需要条件分支和并行处理能力：

In [ ]:
# 高级工作流状态
class AdvancedWorkflowState(TypedDict):
    sensor_readings: Dict[str, float]
    analysis_path: str
    processing_steps: Annotated[List[str], operator.add]
    parallel_results: Dict[str, Any]
    final_decision: str
    confidence_level: float

class AdvancedPHMWorkflow:
    """高级PHM工作流，支持条件分支和并行处理"""
    
    def __init__(self):
        self.workflow = self._build_advanced_workflow()
    
    def _build_advanced_workflow(self) -> StateGraph:
        """构建高级工作流"""
        
        def data_validation_node(state: AdvancedWorkflowState) -> Dict[str, Any]:
            """数据验证节点"""
            readings = state["sensor_readings"]
            
            # 检查数据质量
            missing_sensors = []
            required_sensors = ["temperature", "vibration", "pressure"]
            
            for sensor in required_sensors:
                if sensor not in readings or readings[sensor] is None:
                    missing_sensors.append(sensor)
            
            if missing_sensors:
                analysis_path = "simple_analysis"  # 缺少数据，使用简单分析
                step_msg = f"数据不完整（缺少: {missing_sensors}），使用简单分析"
            else:
                analysis_path = "full_analysis"  # 数据完整，使用完整分析
                step_msg = "数据完整，使用完整分析流程"
            
            return {
                "analysis_path": analysis_path,
                "processing_steps": [step_msg]
            }
        
        def simple_analysis_node(state: AdvancedWorkflowState) -> Dict[str, Any]:
            """简单分析节点"""
            readings = state["sensor_readings"]
            
            # 基础分析逻辑
            avg_value = sum(v for v in readings.values() if v is not None) / len(readings)
            
            if avg_value > 70:
                decision = "需要注意：传感器读数偏高"
                confidence = 0.6
            else:
                decision = "状态正常"
                confidence = 0.8
            
            return {
                "processing_steps": ["执行简单分析"],
                "final_decision": decision,
                "confidence_level": confidence
            }
        
        def parallel_analysis_node(state: AdvancedWorkflowState) -> Dict[str, Any]:
            """并行分析节点（模拟）"""
            readings = state["sensor_readings"]
            
            # 模拟并行处理的多个分析
            temp_analysis = {
                "result": "正常" if readings.get("temperature", 0) < 80 else "异常",
                "score": 0.9 if readings.get("temperature", 0) < 80 else 0.3
            }
            
            vibration_analysis = {
                "result": "正常" if readings.get("vibration", 0) < 5 else "异常",
                "score": 0.85 if readings.get("vibration", 0) < 5 else 0.2
            }
            
            pressure_analysis = {
                "result": "正常" if readings.get("pressure", 0) > 1.5 else "异常",
                "score": 0.9 if readings.get("pressure", 0) > 1.5 else 0.4
            }
            
            parallel_results = {
                "temperature": temp_analysis,
                "vibration": vibration_analysis,
                "pressure": pressure_analysis
            }
            
            return {
                "processing_steps": ["执行并行分析（温度、振动、压力）"],
                "parallel_results": parallel_results
            }
        
        def decision_fusion_node(state: AdvancedWorkflowState) -> Dict[str, Any]:
            """决策融合节点"""
            parallel_results = state["parallel_results"]
            
            # 融合多个分析结果
            total_score = 0
            abnormal_count = 0
            
            for sensor, analysis in parallel_results.items():
                total_score += analysis["score"]
                if analysis["result"] == "异常":
                    abnormal_count += 1
            
            avg_confidence = total_score / len(parallel_results)
            
            if abnormal_count >= 2:
                decision = "多个传感器异常，建议立即检修"
            elif abnormal_count == 1:
                decision = "单个传感器异常，建议密切监控"
            else:
                decision = "所有传感器正常"
            
            return {
                "processing_steps": [f"决策融合完成，异常传感器数量: {abnormal_count}"],
                "final_decision": decision,
                "confidence_level": avg_confidence
            }
        
        # 路由决策函数
        def route_analysis(state: AdvancedWorkflowState) -> str:
            """根据数据质量选择分析路径"""
            return state["analysis_path"]
        
        def should_fuse_results(state: AdvancedWorkflowState) -> str:
            """判断是否需要结果融合"""
            if state.get("parallel_results"):
                return "fusion"
            else:
                return "end"
        
        # 构建图
        workflow = StateGraph(AdvancedWorkflowState)
        
        # 添加节点
        workflow.add_node("validate", data_validation_node)
        workflow.add_node("simple_analysis", simple_analysis_node)
        workflow.add_node("parallel_analysis", parallel_analysis_node)
        workflow.add_node("fusion", decision_fusion_node)
        
        # 设置入口
        workflow.set_entry_point("validate")
        
        # 条件路由
        workflow.add_conditional_edges(
            "validate",
            route_analysis,
            {
                "simple_analysis": "simple_analysis",
                "full_analysis": "parallel_analysis"
            }
        )
        
        # 简单分析直接结束
        workflow.add_edge("simple_analysis", END)
        
        # 并行分析后进行融合
        workflow.add_conditional_edges(
            "parallel_analysis",
            should_fuse_results,
            {
                "fusion": "fusion",
                "end": END
            }
        )
        
        workflow.add_edge("fusion", END)
        
        return workflow.compile()
    
    def run_analysis(self, sensor_readings: Dict[str, float]) -> Dict[str, Any]:
        """运行高级分析"""
        initial_state = {
            "sensor_readings": sensor_readings,
            "analysis_path": "",
            "processing_steps": ["高级工作流启动"],
            "parallel_results": {},
            "final_decision": "",
            "confidence_level": 0.0
        }
        
        result = self.workflow.invoke(initial_state)
        return result

# 测试高级工作流
print("🚀 创建高级PHM工作流")
advanced_workflow = AdvancedPHMWorkflow()

# 测试不同场景
test_scenarios = [
    {
        "name": "完整数据 - 正常状态",
        "data": {"temperature": 70, "vibration": 3.0, "pressure": 2.2}
    },
    {
        "name": "完整数据 - 多项异常",
        "data": {"temperature": 85, "vibration": 6.5, "pressure": 1.0}
    },
    {
        "name": "不完整数据",
        "data": {"temperature": 75, "pressure": 2.1}  # 缺少振动数据
    }
]

for scenario in test_scenarios:
    print(f"\n{'='*60}")
    print(f"🧪 测试场景: {scenario['name']}")
    print(f"📊 输入数据: {scenario['data']}")
    
    result = advanced_workflow.run_analysis(scenario['data'])
    
    print(f"🔄 处理路径: {result['analysis_path']}")
    print(f"📝 处理步骤: {result['processing_steps']}")
    print(f"🎯 最终决策: {result['final_decision']}")
    print(f"📈 置信度: {result['confidence_level']:.2f}")
    
    if result.get('parallel_results'):
        print(f"🔄 并行分析结果:")
        for sensor, analysis in result['parallel_results'].items():
            print(f"  {sensor}: {analysis['result']} (得分: {analysis['score']:.2f})")

---

## 3. Router模式 - 智能路由 🔀

### 3.1 基础Router实现

Router模式用于智能地选择不同的处理路径：

In [ ]:
from abc import ABC, abstractmethod
from enum import Enum

class AnalysisType(Enum):
    QUICK = "quick"
    STANDARD = "standard"
    DETAILED = "detailed"
    EMERGENCY = "emergency"

class BaseRouter(ABC):
    """路由器基类"""
    
    @abstractmethod
    def route(self, data: Dict[str, Any]) -> AnalysisType:
        """路由决策"""
        pass

class RuleBasedRouter(BaseRouter):
    """基于规则的路由器"""
    
    def __init__(self):
        # 定义阈值
        self.emergency_thresholds = {
            "temperature": 90,
            "vibration": 8.0,
            "pressure": 0.5
        }
        
        self.detailed_thresholds = {
            "temperature": 80,
            "vibration": 5.0,
            "pressure": 1.2
        }
    
    def route(self, data: Dict[str, Any]) -> AnalysisType:
        """基于规则的路由决策"""
        sensor_data = data.get("sensor_data", {})
        
        # 检查紧急情况
        for sensor, threshold in self.emergency_thresholds.items():
            value = sensor_data.get(sensor, 0)
            if (sensor == "pressure" and value < threshold) or \
               (sensor != "pressure" and value > threshold):
                return AnalysisType.EMERGENCY
        
        # 检查是否需要详细分析
        detailed_triggers = 0
        for sensor, threshold in self.detailed_thresholds.items():
            value = sensor_data.get(sensor, 0)
            if (sensor == "pressure" and value < threshold) or \
               (sensor != "pressure" and value > threshold):
                detailed_triggers += 1
        
        if detailed_triggers >= 2:
            return AnalysisType.DETAILED
        elif detailed_triggers == 1:
            return AnalysisType.STANDARD
        else:
            return AnalysisType.QUICK

class LLMBasedRouter(BaseRouter):
    """基于LLM的智能路由器"""
    
    def __init__(self, llm_provider: str = "mock"):
        try:
            from modules.llm_providers_unified import create_llm
            self.llm = create_llm(llm_provider, temperature=0.1)
        except ImportError:
            print("⚠️ 无法导入LLM，使用规则回退")
            self.llm = None
            self.rule_router = RuleBasedRouter()
    
    def route(self, data: Dict[str, Any]) -> AnalysisType:
        """基于LLM的智能路由"""
        if self.llm is None:
            return self.rule_router.route(data)
        
        sensor_data = data.get("sensor_data", {})
        user_priority = data.get("priority", "normal")
        
        prompt = f"""
作为PHM系统的路由专家，请根据以下传感器数据和用户优先级，选择最合适的分析类型：

传感器数据: {sensor_data}
用户优先级: {user_priority}

可选的分析类型：
- quick: 快速检查（正常情况）
- standard: 标准分析（轻微异常）
- detailed: 详细分析（多项指标异常）
- emergency: 紧急分析（严重异常）

请只回答分析类型（quick/standard/detailed/emergency），不要其他解释。
"""
        
        try:
            response = self.llm.invoke(prompt)
            result = response.content.strip().lower()
            
            # 验证LLM响应
            for analysis_type in AnalysisType:
                if analysis_type.value in result:
                    return analysis_type
            
            # 如果LLM响应无效，使用规则回退
            print(f"⚠️ LLM响应无效: {result}，使用规则回退")
            return RuleBasedRouter().route(data)
            
        except Exception as e:
            print(f"❌ LLM路由失败: {e}，使用规则回退")
            return RuleBasedRouter().route(data)

# 路由系统演示
class RouterDemo:
    """路由系统演示"""
    
    def __init__(self):
        self.rule_router = RuleBasedRouter()
        self.llm_router = LLMBasedRouter()
    
    def demo_routing(self):
        """演示不同路由策略"""
        test_cases = [
            {
                "name": "正常状态",
                "data": {
                    "sensor_data": {"temperature": 65, "vibration": 2.5, "pressure": 2.0},
                    "priority": "normal"
                }
            },
            {
                "name": "单项异常",
                "data": {
                    "sensor_data": {"temperature": 82, "vibration": 3.0, "pressure": 2.2},
                    "priority": "normal"
                }
            },
            {
                "name": "多项异常",
                "data": {
                    "sensor_data": {"temperature": 85, "vibration": 6.0, "pressure": 1.8},
                    "priority": "high"
                }
            },
            {
                "name": "紧急情况",
                "data": {
                    "sensor_data": {"temperature": 95, "vibration": 9.0, "pressure": 0.3},
                    "priority": "urgent"
                }
            }
        ]
        
        print("🔀 路由系统演示")
        print(f"{'场景':<15} {'规则路由':<15} {'LLM路由':<15}")
        print("-" * 50)
        
        for case in test_cases:
            rule_result = self.rule_router.route(case["data"])
            llm_result = self.llm_router.route(case["data"])
            
            print(f"{case['name']:<15} {rule_result.value:<15} {llm_result.value:<15}")
            
            # 显示详细信息
            sensor_data = case["data"]["sensor_data"]
            print(f"  📊 传感器: T={sensor_data['temperature']}°C, V={sensor_data['vibration']}, P={sensor_data['pressure']}bar")
            print()

# 运行路由演示
router_demo = RouterDemo()
router_demo.demo_routing()

---

## 4. 实践练习 📝

### 练习：构建您的多节点工作流

结合本章学到的概念，创建一个完整的PHM分析系统：

In [ ]:
# 练习：完善这个集成的PHM系统
class IntegratedPHMSystem:
    """集成的PHM系统 - 请完善实现"""
    
    def __init__(self, llm_provider="mock"):
        self.router = LLMBasedRouter(llm_provider)
        # TODO: 初始化不同类型的分析工作流
        self.workflows = {
            AnalysisType.QUICK: self._create_quick_workflow(),
            AnalysisType.STANDARD: self._create_standard_workflow(),
            AnalysisType.DETAILED: self._create_detailed_workflow(),
            AnalysisType.EMERGENCY: self._create_emergency_workflow()
        }
    
    def _create_quick_workflow(self):
        """创建快速分析工作流 - 请实现"""
        # TODO: 实现快速检查工作流
        def quick_check(state):
            return {"result": "快速检查完成", "confidence": 0.7}
        return quick_check
    
    def _create_standard_workflow(self):
        """创建标准分析工作流 - 请实现"""
        # TODO: 实现标准分析工作流
        def standard_analysis(state):
            return {"result": "标准分析完成", "confidence": 0.8}
        return standard_analysis
    
    def _create_detailed_workflow(self):
        """创建详细分析工作流 - 请实现"""
        # TODO: 实现详细分析工作流
        def detailed_analysis(state):
            return {"result": "详细分析完成", "confidence": 0.9}
        return detailed_analysis
    
    def _create_emergency_workflow(self):
        """创建紧急分析工作流 - 请实现"""
        # TODO: 实现紧急分析工作流
        def emergency_analysis(state):
            return {"result": "紧急分析完成，建议立即停机", "confidence": 0.95}
        return emergency_analysis
    
    def analyze(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """执行完整的PHM分析"""
        # 1. 路由决策
        analysis_type = self.router.route(data)
        
        # 2. 执行相应的工作流
        workflow = self.workflows[analysis_type]
        result = workflow(data)
        
        # 3. 返回完整结果
        return {
            "input_data": data,
            "analysis_type": analysis_type.value,
            "result": result["result"],
            "confidence": result["confidence"]
        }

# 测试集成系统
print("🏭 创建集成PHM系统")
integrated_system = IntegratedPHMSystem()

# 测试不同输入
test_inputs = [
    {
        "sensor_data": {"temperature": 70, "vibration": 3.0, "pressure": 2.0},
        "timestamp": "2024-01-15 10:00:00"
    },
    {
        "sensor_data": {"temperature": 95, "vibration": 8.5, "pressure": 0.5},
        "priority": "urgent"
    }
]

for i, test_input in enumerate(test_inputs, 1):
    print(f"\n🧪 测试 {i}: {test_input}")
    result = integrated_system.analyze(test_input)
    print(f"📊 分析类型: {result['analysis_type']}")
    print(f"🎯 分析结果: {result['result']}")
    print(f"📈 置信度: {result['confidence']:.2f}")

print("\n💡 挑战: 尝试实现更复杂的工作流和状态管理！")

---

## 5. 本章总结 🎯

### 您已经学会了：

✅ **状态管理** - TypedDict和Annotated类型的使用  
✅ **LangGraph工作流** - 多节点、条件分支、并行处理  
✅ **Router模式** - 规则路由和LLM智能路由  
✅ **系统集成** - 将多个组件组合成完整系统  

### 关键概念回顾：

- **Annotated[List, operator.add]** - 自动累加列表
- **条件路由** - 根据状态动态选择执行路径
- **并行处理** - 多个分析同时进行
- **Router模式** - 智能选择处理策略

### 下一步：

在 **Part 3** 中，您将学习：
- 🤖 **ReAct模式** - 推理-行动循环的高级Agent
- 👥 **多Agent协作** - Agent团队的设计和管理
- 🛠️ **工具集成** - Agent如何使用外部工具

---

## 🎓 准备继续？

<div style="text-align: center; margin: 20px;">
    <a href="../Part3_Agent_Architectures/03_Complete_Tutorial.ipynb" 
       style="background: #FF9800; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px;">
       🤖 继续 Part 3: Agent Architectures
    </a>
</div>